In [1]:
import modelWin_env as modelWin
import discrete_MDP_transition_based_rewards as MDP
import generate_dataset
import numpy as np
from estimators import *

In [12]:
# Compare results with the R implementation
n = 100; horizon = 5; n_columns = 8; n_states = 3; n_actions = 2
D = np.genfromtxt('R/data.csv', delimiter=',').reshape((n, horizon, n_columns), order='F')
# Need to subtract 1 from the states and actions indices
D[:, :, 0] -= 1
D[:, :, 1] -= 1
Q_hat = np.genfromtxt('R/Q_hat.csv', delimiter=',').reshape((horizon, n_states, n_actions), order='F')
V_hat = np.genfromtxt('R/V_hat.csv', delimiter=',')
print(IPS(D))
print(WIPS(D))
print(stepIPS(D))
print(stepWIPS(D))
print(DR(D, Q_hat, V_hat))
print(LTMLE(D, Q_hat, V_hat, modelWin.evaluation_policy_matrix))

-0.28125444079999995
-0.4380213758338713
0.02188125109999997
-0.04880679305197754
0.1354181801599999
0.03222255559277132


In [12]:
# Now, try the estimators on an environment
# First, instantiate an environmenent
s0 = 0
env = MDP.discrete_MDP_transition_based_rewards(modelWin.state_transition_matrix,
                                                modelWin.transition_based_rewards)
env.reset(s0)

In [13]:
# Generate some data
n = 10000; horizon = 5
D = generate_dataset.generate_dataset(env, s0, horizon, n,
                                      modelWin.behavior_action_matrix,
                                      modelWin.evaluation_policy_matrix)
D.shape
# columns of D: s, a, r, pi_b, pi_e, rho_t

(10000, 5, 6)

In [14]:
# Get the true value of the policy value
Q0, V0 = env.get_true_Q_and_V_functions(modelWin.evaluation_policy_matrix, 1, horizon)
V0[0,:]

array([0.276, 0.184, 0.184])

In [15]:
print(IPS(D))
print(stepIPS(D))
print(WIPS(D))
print(stepWIPS(D))
print(DR(D, Q0, V0))
print(WDR(D, Q0, V0))

0.288666876295068
0.30934108673038707
0.29705044158721056
0.3161740955037919
0.32034237330683163
0.3211496515689108
